In [1]:
pip install -U git+https://github.com/QuantFreedom1022/quantfreedom@dev

  Cloning https://github.com/QuantFreedom1022/quantfreedom (to revision dev) to c:\users\new user\appdata\local\temp\pip-req-build-_xfffgt1
  Resolved https://github.com/QuantFreedom1022/quantfreedom to commit 2633753b2478faccaee59da48ab230c5187590d2
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/QuantFreedom1022/quantfreedom 'C:\Users\New User\AppData\Local\Temp\pip-req-build-_xfffgt1'
  Running command git checkout -b dev --track origin/dev
  branch 'dev' set up to track 'origin/dev'.
  Switched to a new branch 'dev'


In [2]:
pip install vectorbt

In [3]:
#installation of Yfinance 
!pip install yfinance

In [4]:
import os
import sys
sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

import quantfreedom as qf
import vectorbt as vbt
import yfinance as yf
import numpy as np
import pandas as pd
from dash import Dash, html, dcc
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
import talib
from talib.abstract import Function
from talib import abstract

from quantfreedom.backtester.indicators.talib_ind import from_talib, talib_func_list_website_link, talib_ind_info
from quantfreedom.backtester.evaluators.evaluators import eval_is_below, eval_is_above, combine_evals
from quantfreedom._typing import (
    pdFrame, pdSeries)

In [5]:
# Fetch historical data

# determine variable 'symbol' for fetching many tickets simultaneously (at same time)
# symbol = [
#     'BTC-USD', 'ETH-USD', 'XRP-USD', 'BCH-USD', 'LTC-USD'#,
# ]
# """    'BNB-USD', 'EOS-USD', 'XLM-USD', 'XMR-USD', 'ADA-USD',
#     'QBIT-USD', 'TSLA', 'AMZN', 'AAPL', 'DGTX-USD', 'BTG-USD', 
#     'LINK-USD', 'DOGE-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD',
#     'TON-USD', 'OKB-USD', 'GLD', 'GIS', 'PG', 'MRK', 'AMT', 'BMY',
#     'CVX', 'S', 'CVS', 'HD', 'PUK'
# #]
# """


symbol = ['BTC-USD']

# "Attach" dataset prices (dataframe) to variable ohlcv and at the same time download dataset from Yahoo Finance 
ohlcv = yf.download(symbol, interval="5m", period="60d")

# "Attach" only closing prices from dataset to variable 'price'
prices = ohlcv['Close']

# "Attach" only High prices from dataset to variable 'high_price'
high_price = ohlcv['High']

# Same for low
low_price = ohlcv['Low']

[*********************100%***********************]  1 of 1 completed


In [6]:
#Renaming columns for TA-Lib
ohlcv.rename(columns={'Open': 'open', 'High': 'high', 'Low': 'low', 'Adj Close': 'close', 'Volume': 'volume'}, inplace=True)

In [7]:
ohlcv.head(5)

,open,high,low,Close,close,volume
Datetime,,,,,,
2023-01-30 00:00:00+00:00,23774.648438,23774.648438,23756.929688,23756.929688,23756.929688,0
2023-01-30 00:05:00+00:00,23757.748047,23769.246094,23757.748047,23769.246094,23769.246094,49373184
2023-01-30 00:10:00+00:00,23780.179688,23789.347656,23780.179688,23789.347656,23789.347656,64131072
2023-01-30 00:15:00+00:00,23785.736328,23785.736328,23762.759766,23762.759766,23762.759766,0
2023-01-30 00:20:00+00:00,23748.007812,23748.007812,23732.167969,23732.167969,23732.167969,71106560


In [8]:
prices.head(5)

Datetime
2023-01-30 00:00:00+00:00    23756.929688
2023-01-30 00:05:00+00:00    23769.246094
2023-01-30 00:10:00+00:00    23789.347656
2023-01-30 00:15:00+00:00    23762.759766
2023-01-30 00:20:00+00:00    23732.167969
Name: Close, dtype: float64

In [10]:
 
rsi_ind = from_talib(
    func_name='rsi',
    df_prices=ohlcv,
    cart_product=False,
    combos=False,
    timeperiod=30,
)


 
# talib.abstract.RSI(prices)


In [11]:
# Define Divergence
def divergence(
    # using price set as starting point to calculate divergence
    price_set,
    # using last 14 bars/days period window for RSI indicator. This is just a template and can be changed when we will execute .run function in next stage.
    rsi_window,
    # using last 1 bar/day period window for fast Moving Average indicator. This is just a template and can be changed when we will execute .run function in next stage.
    fast_ma_window,

    # using last 50 bars/days period window for fast Moving Average indicator.  This is just a template and can be changed when we will execute .run function in next stage.
        slow_ma_window,):

    slow_ma = from_talib(
        func_name='MA',
        df_prices=ohlcv,
        cart_product=False,
        combos=False,
        timeperiod=slow_ma_window,
    )
    fast_ma = from_talib(
        func_name='MA',
        df_prices=ohlcv,
        cart_product=False,
        combos=False,
        timeperiod=fast_ma_window,
    )
    rsi_indicator = from_talib(
        func_name='MA',
        df_prices=ohlcv,
        cart_product=False,
        combos=False,
        timeperiod=rsi_window,
    ).squeeze()

# Calculate the Divergence indicator. Arithmetics behind indicators is black box for me and consist of pure mathematics
    bullish_divergence = ((price_set.diff() < 0) & (rsi_indicator.diff() > 0)).to_numpy()

    bearish_divergence = ((price_set.diff() < 0) &
                          (rsi_indicator.diff() < 0)).to_numpy()

    # Determine UpTrend (fast MA above slow MA)
    trend_up = eval_is_above(
        ind_data=fast_ma,
        ind_results=slow_ma.squeeze(),
    )
    trend_down = eval_is_below(
        ind_data=fast_ma,
        ind_results=slow_ma.squeeze(),
    )

    # Convert everything to Numpy array
    trend_up = trend_up.to_numpy().flatten()
    trend_down = trend_down.to_numpy().flatten()

    # Combine all previous indicators into the one
    # We need that if bullish_divergence (trend_up), True PLUS fast MA above slow MA, then set 1, everything else - 0. If bearish_divergence True, PLUS fast MA above slow MA (trend_down), then set -1
    indicator_buy = np.logical_and(bullish_divergence == True,trend_up == True)
    indicator_buy = np.where(indicator_buy == True, 1, 0)
    indicator_sell = np.logical_and(bearish_divergence == True,trend_down == True)
    indicator = np.where(indicator_sell == True, -1, indicator_buy)


    return indicator

atr_ind = from_talib(
    func_name='atr',
    df_prices=ohlcv,
    cart_product=False,
    combos=False,
    timeperiod=14,
)

talib.SMA(atr_ind.values.flatten(), timeperiod=50)



array([        nan,         nan,         nan, ..., 30.55450938,
       31.10194119, 31.57429974])

In [12]:
pf = vbt.Portfolio.from_signals(ohlcv, divergence(indicator_buy), divergence(indicator_sell))

# Output
print(pf.total_return())
print(pf.stats())

NameError: name 'indicator_buy' is not defined

In [ ]:
divergence(
    price_set=ohlcv.close,
    rsi_window=15,
    fast_ma_window=10,
    slow_ma_window=40,
)

In [13]:
atr_ind = from_talib(
    func_name='atr',
    df_prices=ohlcv,
    cart_product=False,
    combos=False,
    timeperiod=14,
)

In [14]:
atr_ind

ATR_timeperiod,14
Datetime,
2023-01-30 00:00:00+00:00,NaN
2023-01-30 00:05:00+00:00,NaN
2023-01-30 00:10:00+00:00,NaN
2023-01-30 00:15:00+00:00,NaN
2023-01-30 00:20:00+00:00,NaN
...,...
2023-03-30 14:20:00+00:00,51.749028
2023-03-30 14:25:00+00:00,49.871446
2023-03-30 14:30:00+00:00,52.675411


In [15]:
talib.SMA(atr_ind.values.flatten(), timeperiod=50)


array([        nan,         nan,         nan, ..., 30.55450938,
       31.10194119, 31.57429974])

In [16]:
(1000 / ohlcv.close.values.flatten())

array([0.04209298, 0.04207117, 0.04203562, ..., 0.03516366, 0.03518946,
       0.0351739 ])

In [17]:
talib.SMA(atr_ind.values.flatten(), timeperiod=50) * (1000 / ohlcv.close.values.flatten())

array([       nan,        nan,        nan, ..., 1.07440842, 1.09446053,
       1.11059116])

In [18]:
fast_ma = vbt.MA.run(price_set, fast_ma_window, short_name='fast')
#printing tpe of bearish_divergence 
print("printing tpe of bearish_divergence dataset:")
print(type(bearish_divergence))
# Get slow Moving Average (MA) of the prices

atr = vbt.ATR.run(high_price, low_price, price, atr_window)
#atr = atr.atr.to_numpy()

atr = atr.atr.to_numpy()
##print(type(atr))

# Calculate adaptive/dynamically changing value based on ATR and current price 
atr_results = atr * atr_scaler / price #* rsi_window

slow_ma = vbt.MA.run(atr_results, slow_ma_window, short_name='slow')
print("printing tpe of slow_ma dataset:")
print(type(slow_ma))

NameError: name 'price_set' is not defined

In [19]:
fig = go.Figure()
fig.add_scatter(
    x=atr_ind.index.to_list(),
    y=ohlcv.close.values.flatten(),
    mode='lines',
    marker=dict(color='red'),
    name='ATR'

)
fig.add_scatter(
    x=rsi_ind.index.to_list(),
    y=talib.SMA(atr_ind.values.flatten(), timeperiod=100) * 1000 / ohlcv.close.values.flatten(),
    mode='lines',
    marker=dict(color='blue'),
    name='SMA'
)
fig.show()


In [20]:
atr_ind.values.flatten()

array([        nan,         nan,         nan, ..., 52.67541108,
       53.50230694, 50.75618792])

In [21]:
atr_ind

ATR_timeperiod,14
Datetime,
2023-01-30 00:00:00+00:00,NaN
2023-01-30 00:05:00+00:00,NaN
2023-01-30 00:10:00+00:00,NaN
2023-01-30 00:15:00+00:00,NaN
2023-01-30 00:20:00+00:00,NaN
...,...
2023-03-30 14:20:00+00:00,51.749028
2023-03-30 14:25:00+00:00,49.871446
2023-03-30 14:30:00+00:00,52.675411


In [22]:
atr_ma = from_talib(
    func_name='MA',
    df_prices=None,
    cart_product=False,
    combos=False,
    timeperiod=50,
    price=atr_ind.values.flatten()
)

ValueError: You need to send either a dataframe of prices 'df_prices' or a dataframe with multindexed values 'user_ind_df'

In [23]:
talib_ind_info('stoch')

{'name': 'STOCH',
 'group': 'Momentum Indicators',
 'display_name': 'Stochastic',
 'function_flags': None,
 'input_names': OrderedDict([('prices', ['high', 'low', 'close'])]),
 'parameters': OrderedDict([('fastk_period', 5),
              ('slowk_period', 3),
              ('slowk_matype', 0),
              ('slowd_period', 3),
              ('slowd_matype', 0)]),
 'output_flags': OrderedDict([('slowk', ['Dashed Line']),
              ('slowd', ['Dashed Line'])]),
 'output_names': ['slowk', 'slowd']}

In [24]:
close = np.random.random(100)
open = np.random.random(100)
high = np.random.random(100)

In [25]:
Function('sma')(np.random.random(10000), 50)

array([       nan,        nan,        nan, ..., 0.52824684, 0.5339133 ,
       0.53869738])

In [26]:
_, hey = talib.STOCH(close, open, high, *(10,15))
hey

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
       64.32393177, 66.6870501 , 66.76191257, 67.34609321, 67.92898945,
       69.52298806, 70.03183008, 67.92225298, 64.76211244, 63.20264711,
       62.31242218, 60.45357047, 58.38268585, 54.53269058, 53.00963536,
       51.98864836, 53.69144074, 53.81293356, 52.00207144, 50.41930072,
       50.82101968, 52.75082329, 55.00529861, 58.6382706 , 62.5305085 ,
       65.97141759, 69.58523045, 73.05078802, 77.93748556, 79.94697068,
       80.78171021, 78.87168798, 80.7587544 , 83.80531585, 86.08908011,
       84.42391536, 80.99154726, 77.17170828, 72.38949569, 68.77996999,
       64.76740279, 62.36100223, 58.19883712, 54.9718197 , 51.02